In [8]:
import os
import pandas as pd
from moviepy.video.io.VideoFileClip import VideoFileClip
import time

In [11]:
def get_file_path(file_type, extensions):
    """Detect a file by its type and ask the user if it is the correct file."""
    script_dir = os.getcwd()
    matching_files = []

    # Scan the script directory for files with the given extensions
    for root, _, files in os.walk(script_dir):
        for file in files:
            if file.lower().endswith(tuple(extensions)):
                matching_files.append(os.path.join(root, file))
    
    if not matching_files:
        print(f"No {file_type} files found in the script directory.")
        return input(f"Enter the path to your {file_type} file: ")
    
    # If multiple files are found, list them and ask the user to choose
    if len(matching_files) > 1:
        print(f"Multiple {file_type} files found:")
        for idx, file in enumerate(matching_files, 1):
            print(f"{idx}. {file}")
        selected_file = int(input(f"Select the correct {file_type} file (1-{len(matching_files)}): "))
        return matching_files[selected_file - 1]
    
    # If exactly one file is found, ask the user to confirm
    selected_file = matching_files[0]
    user_input = input(f"Found {file_type} file: {selected_file}. Is this the correct file? (y/n): ")
    if user_input.lower() == 'y':
        return selected_file
    else:
        return input(f"Enter the path to your {file_type} file: ")


In [13]:
def time_to_seconds(time_str):
    """Convert time format HH:MM:SS,mmm to seconds."""
    # Split the time string into parts before and after the comma
    time_parts = time_str.split(",")
    # Convert the main time portion HH:MM:SS
    h, m, s = map(int, time_parts[0].split(":"))
    # Add milliseconds if present
    milliseconds = 0
    if len(time_parts) > 1:
        milliseconds = int(time_parts[1]) / 1000
    return h * 3600 + m * 60 + s + milliseconds

def seconds_to_time(seconds):
    """Convert seconds to time format HH:MM:SS,mmm."""
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    ms = int((seconds - int(seconds)) * 1000)
    return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"

In [17]:
def extract_srt_for_clip(srt_content, clip_start_time_seconds):
    """Extract and adjust subtitles for a specific clip."""
    srt_lines = srt_content.split("\n\n")
    extracted_lines = []

    for entry in srt_lines:
        # Split the entry into lines, handling multi-line subtitles
        parts = entry.strip().split("\n")
        
        # Skip empty entries or entries with too few parts
        if len(parts) < 3:
            continue
        
        # First line is the subtitle index
        index = parts[0]
        
        # Second line contains the time range
        time_range = parts[1]
        
        # Remaining lines are the subtitle text (can be multiple lines)
        text = "\n".join(parts[2:])
        
        start_srt, end_srt = time_range.split(" --> ")

        start_srt_seconds = time_to_seconds(start_srt)
        end_srt_seconds = time_to_seconds(end_srt)

        # Adjust the subtitle start and end times relative to the clip start time
        adjusted_start_time = start_srt_seconds - clip_start_time_seconds
        adjusted_end_time = end_srt_seconds - clip_start_time_seconds

        # Only keep subtitles that are within the current clip's time range
        if adjusted_start_time >= 0:
            adjusted_start_time_str = seconds_to_time(adjusted_start_time)
            adjusted_end_time_str = seconds_to_time(adjusted_end_time)

            extracted_lines.append(f"{index}\n{adjusted_start_time_str} --> {adjusted_end_time_str}\n{text}\n")

    return "\n\n".join(extracted_lines)

In [19]:
def split_video(video_path, chapters_file, srt_file, output_dir):
    # Read chapters from Excel
    chapters = pd.read_excel(chapters_file)
    
    # Check if output directory exists, create if not
    os.makedirs(output_dir, exist_ok=True)
    
    # Load video file
    video = VideoFileClip(video_path)
    
    # Load the SRT file
    with open(srt_file, 'r', encoding='utf-8') as srt:
        srt_content = srt.read()

    # Iterate through chapters and split the video
    for index, row in chapters.iterrows():
        start_time = time_to_seconds(row["Start Time"])
        end_time = time_to_seconds(row["End Time"])
        title = row["Title"]
        
        # Extract clip
        clip = video.subclipped(start_time, end_time)  # Fixed method name to subclip
        
        # Clean title for a valid filename
        filename = f"{index + 1:02d}_{title.replace('/', '-').replace(':', ' -')}.mp4"
        output_path = os.path.join(output_dir, filename)
        
        # Write the clip to a file
        print(f"Processing: {title} -> {filename}")
        clip.write_videofile(output_path, codec="libx264", audio_codec="aac")
        
        # Extract corresponding subtitles for the clip
        srt_clip = extract_srt_for_clip(srt_content, start_time)  # Only pass the start time
        srt_output_path = os.path.join(output_dir, f"{filename.replace('.mp4', '.srt')}")
        
        # Save the SRT file for the clip
        with open(srt_output_path , 'w', encoding='utf-8') as srt_file:
            srt_file.write(srt_clip)

    print("Video splitting completed.")

In [21]:
if __name__ == "__main__":
    # Automatically detect files based on type
    video_path = get_file_path("video", [".mp4", ".mkv", ".avi"])  # You can add more video extensions
    chapters_file = get_file_path("Chapters", [".xlsx", ".xls"])  # You can add more extensions for chapters files
    srt_file = get_file_path("SRT", [".srt"])  # Looking for .srt files
    
    output_dir = os.path.join(os.getcwd(), 'new_videos')  # Append 'new_videos' to output directory path
    
    # Split the video (call your existing split_video function here)
    split_video(video_path, chapters_file, srt_file, output_dir)

Found video file: F:\Github2\Helping-Study-Tools\YouSplit\Master Business English - Learn Professional Business English with 5 Hour Masterclass.mp4. Is this the correct file? (y/n):  y
Found Chapters file: F:\Github2\Helping-Study-Tools\YouSplit\Chapters.xlsx. Is this the correct file? (y/n):  y
Found SRT file: F:\Github2\Helping-Study-Tools\YouSplit\Master Business English  Learn Professional Business English with 5 Hour Masterclass.srt. Is this the correct file? (y/n):  y


{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '1', 'compatible_brands': 'isomiso2avc1mp41', 'creation_time': '2025-03-13T05:35:38.000000Z'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [854, 480], 'bitrate': 203, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'creation_time': '2025-03-13T05:35:38.000000Z', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'creation_time': '2024-08-27T13:54:54.000000Z', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 18645.11, 'bitrate': 335, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_numb

MoviePy - Done.
MoviePy - Writing video F:\Github2\Helping-Study-Tools\YouSplit\new_videos\01_English Phrases for Virtual Meetings.mp4



MoviePy - Done !
MoviePy - video ready F:\Github2\Helping-Study-Tools\YouSplit\new_videos\01_English Phrases for Virtual Meetings.mp4
Processing: 100 English Phrases for Sales -> 02_100 English Phrases for Sales.mp4
MoviePy - Building video F:\Github2\Helping-Study-Tools\YouSplit\new_videos\02_100 English Phrases for Sales.mp4.
MoviePy - Writing audio in 02_100 English Phrases for SalesTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video F:\Github2\Helping-Study-Tools\YouSplit\new_videos\02_100 English Phrases for Sales.mp4



...

MoviePy - Done !
MoviePy - video ready F:\Github2\Helping-Study-Tools\YouSplit\new_videos\24_Mortgages in English.mp4
Processing: Buying a Car in English -> 25_Buying a Car in English.mp4
MoviePy - Building video F:\Github2\Helping-Study-Tools\YouSplit\new_videos\25_Buying a Car in English.mp4.
MoviePy - Writing audio in 25_Buying a Car in EnglishTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video F:\Github2\Helping-Study-Tools\YouSplit\new_videos\25_Buying a Car in English.mp4



MoviePy - Done !
MoviePy - video ready F:\Github2\Helping-Study-Tools\YouSplit\new_videos\25_Buying a Car in English.mp4
Processing: Intermediate Banking Terms -> 26_Intermediate Banking Terms.mp4
MoviePy - Building video F:\Github2\Helping-Study-Tools\YouSplit\new_videos\26_Intermediate Banking Terms.mp4.
MoviePy - Writing audio in 26_Intermediate Banking TermsTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video F:\Github2\Helping-Study-Tools\YouSplit\new_videos\26_Intermediate Banking Terms.mp4



MoviePy - Done !
MoviePy - video ready F:\Github2\Helping-Study-Tools\YouSplit\new_videos\26_Intermediate Banking Terms.mp4
Video splitting completed.
